In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import websocket
import json
import threading
from datetime import datetime, timedelta
from collections import deque
import os
from dotenv import load_dotenv

Websocket

In [2]:
websocket_data = []

def on_message(ws, message):
    data = json.loads(message)
    wdata = data['k']
    timestamp = pd.to_datetime(wdata['t'], unit='ms')
    price = float(wdata['c'])
    websocket_data.append((timestamp, price))
    
def on_open(ws):
    payload = {"method": "SUBSCRIBE",
            "params": ["btcusdt@kline_1s"],
               "id": 1}
    ws.send(json.dumps(payload))
    time = threading.Timer(1440, ws.close)
    time.start()
def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status, close_message):
    print("WebSocket closed")

def get_websocket_data():
    ws = websocket.WebSocketApp("wss://stream.binance.com:9443/ws",
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close )
    ws.run_forever()

thread = threading.Thread(target=get_websocket_data, daemon = True)
thread.start()
thread.join()

def data_metrics_analysis(df):
    try:
        df['time_m'] = df['timestamp'].dt.floor('min')
        df = df[df["time_m"] > min(df["time_m"])]
        
        dp_df = df["time_m"].value_counts().sort_index().reset_index()
        dp_df.columns = ["time_m", "Volume"]
        dp_df = dp_df.head(10)

        dp_df["Highest"] = dp_df["Volume"].cummax()
        dp_df["Lowest"] = dp_df["Volume"].cummin()
        dp_df["Mean"] = dp_df["Volume"].expanding().mean()
        return dp_df
    except Exception as e:
        raise Exception(f"Data metrics calculation failure")

def moving_average(df):
    try:
        moving_avg = df.groupby("time_m")["price"].mean().reset_index()
        moving_avg["Moving_avg_10"] = moving_avg["price"].rolling(window=10).mean()
        moving_avg["Moving_avg_20"] = moving_avg["price"].rolling(window=20).mean()
        moving_avg["Moving_avg_price_10"] = moving_avg["price"] + moving_avg["Moving_avg_10"]
        return moving_avg
    except Exception as e:
        raise Exception(f"Moving average calculation failure")
    
main_df = pd.DataFrame(websocket_data, columns=["timestamp", "price"])
metrics_df = data_metrics_analysis(main_df)
moving_avg_df = moving_average(main_df)


Error: 'k'
WebSocket closed


Highest, Lowest, Mean, Volume of data points per minute

In [8]:
metrics_df

,time_m,Volume,Highest,Lowest,Mean
0,2025-06-16 16:19:00,60,60,60,60.0
1,2025-06-16 16:20:00,60,60,60,60.0
2,2025-06-16 16:21:00,60,60,60,60.0
3,2025-06-16 16:22:00,60,60,60,60.0
4,2025-06-16 16:23:00,60,60,60,60.0
5,2025-06-16 16:24:00,60,60,60,60.0
6,2025-06-16 16:25:00,60,60,60,60.0
7,2025-06-16 16:26:00,60,60,60,60.0
8,2025-06-16 16:27:00,60,60,60,60.0
9,2025-06-16 16:28:00,60,60,60,60.0


Moving Average

In [4]:
moving_avg_df

,time_m,price,Moving_avg_10,Moving_avg_20,Moving_avg_price_10
0,2025-06-16 16:18:00,107666.688437,NaN,NaN,NaN
1,2025-06-16 16:19:00,107638.026167,NaN,NaN,NaN
2,2025-06-16 16:20:00,107649.078833,NaN,NaN,NaN
3,2025-06-16 16:21:00,107666.301333,NaN,NaN,NaN
4,2025-06-16 16:22:00,107655.127167,NaN,NaN,NaN
5,2025-06-16 16:23:00,107665.636000,NaN,NaN,NaN
6,2025-06-16 16:24:00,107670.952000,NaN,NaN,NaN
7,2025-06-16 16:25:00,107649.488667,NaN,NaN,NaN
8,2025-06-16 16:26:00,107629.141667,NaN,NaN,NaN
9,2025-06-16 16:27:00,107619.384667,107650.982494,NaN,215270.367160


API

In [5]:
load_dotenv()

def load_klines(symbol, interval, lookback, client):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        return klines
    except Exception as e:
        raise Exception(f"API call failure")

def transform_data(klines):
    try:
        selected_data = [(i[0], float(i[4])) for i in klines]
        raw_df = pd.DataFrame(selected_data, columns=["timestamp", "price"])
        raw_df["timestamp"] = pd.to_datetime(raw_df["timestamp"], unit = "ms")
        return raw_df
    except Exception as e:
        raise Exception(f"Data transformation failure")

api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_API_SECRET")

if not api_key or not api_secret:
    raise Exception("Missing API credentials.")

client = Client(api_key, api_secret)
symbol = "BTCUSDT"
interval = "1s"
lookback = "1700 sec ago UTC"

data = load_klines(symbol, interval, lookback, client)
raw_df = transform_data(data)
metrics_api_df = data_metrics_analysis(raw_df)
moving_avg_api_df = moving_average(raw_df)

Highest, Lowest, Mean, Volume of data points per minute

In [6]:
metrics_api_df

,time_m,Volume,Highest,Lowest,Mean
0,2025-06-16 16:15:00,60,60,60,60.0
1,2025-06-16 16:16:00,60,60,60,60.0
2,2025-06-16 16:17:00,60,60,60,60.0
3,2025-06-16 16:18:00,60,60,60,60.0
4,2025-06-16 16:19:00,60,60,60,60.0
5,2025-06-16 16:20:00,60,60,60,60.0
6,2025-06-16 16:21:00,60,60,60,60.0
7,2025-06-16 16:22:00,60,60,60,60.0
8,2025-06-16 16:23:00,60,60,60,60.0
9,2025-06-16 16:24:00,60,60,60,60.0


Moving Average

In [7]:
moving_avg_api_df

,time_m,price,Moving_avg_10,Moving_avg_20,Moving_avg_price_10
0,2025-06-16 16:14:00,107665.707308,NaN,NaN,NaN
1,2025-06-16 16:15:00,107646.601833,NaN,NaN,NaN
2,2025-06-16 16:16:00,107668.793500,NaN,NaN,NaN
3,2025-06-16 16:17:00,107661.038333,NaN,NaN,NaN
4,2025-06-16 16:18:00,107674.522667,NaN,NaN,NaN
5,2025-06-16 16:19:00,107638.026167,NaN,NaN,NaN
6,2025-06-16 16:20:00,107649.078833,NaN,NaN,NaN
7,2025-06-16 16:21:00,107666.301333,NaN,NaN,NaN
8,2025-06-16 16:22:00,107655.127167,NaN,NaN,NaN
9,2025-06-16 16:23:00,107665.636000,107659.083314,NaN,215324.719314
